In [9]:
from torch.utils.data import DataLoader
from data_collator import DataCollatorForLiltPretraining
import argparse
from pl_lilt_datamodule import LiltPretrainingDataModule
import torch

In [3]:
parser = argparse.ArgumentParser()
parser = LiltPretrainingDataModule.add_model_specific_args(parser)
args = parser.parse_args('')

In [4]:
dm = LiltPretrainingDataModule(args)
dm.prepare_data()

/home/hj36wegi/.conda/envs/adapters/lib/python3.11/site-packages/transformers/models/layoutlmv2/feature_extraction_layoutlmv2.py:30: FutureWarning: The class LayoutLMv2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use LayoutLMv2ImageProcessor instead.
  warnings.warn(
Loading cached processed dataset at /work/scratch/hj36wegi/data/rvl_cdip_processed/train/cache-46290f929498e386_*_of_00032.arrow


Filtering dataset for category : form



Loading cached processed dataset at /work/scratch/hj36wegi/data/rvl_cdip_processed/validation/cache-caf03d03c32e70fc_*_of_00032.arrow
Loading cached processed dataset at /work/scratch/hj36wegi/data/rvl_cdip_processed/test/cache-438a5461cf4b2c7e_*_of_00032.arrow


In [13]:
dataset = dm.dataset['train']
tokenizer = dm.processor.tokenizer

In [6]:
examples = [dataset[i] for i in range(4)]
batch = {k:[ex[k] for ex in examples] for k in examples[0].keys()}

In [7]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'special_tokens_mask', 'bbox'])

In [10]:
seq_len = 512
bs = 4

In [15]:
pad_tokens = {'input_ids':tokenizer.pad_token_id,
              'attention_mask':0,
              'special_tokens_mask':1,
              'bbox':0}
pad_tokens

{'input_ids': 1, 'attention_mask': 0, 'special_tokens_mask': 1, 'bbox': 0}

In [30]:
def pad(examples,key,pad_token_id):
    if key=='bbox':
        output_shape = (bs,seq_len,4)
    else:
        output_shape = (bs,seq_len)
    output = torch.full(output_shape,pad_token_id)
    
    for i,example in enumerate(examples):
        if tokenizer.padding_side == "right":
            output[i, : example.shape[1]] = example
        else:
            output[i, -example.shape[1] :] = example
    return output

In [21]:
collated = {}
for key in batch.keys():
    collated[key] = pad(batch[key],key,pad_tokens[key])

In [24]:
[v.shape for k,v in collated.items()]

[torch.Size([4, 512]),
 torch.Size([4, 512]),
 torch.Size([4, 512]),
 torch.Size([4, 512, 4])]

In [27]:
tokenizer.padding_side, tokenizer.

'right'

In [25]:
def collate_fn(examples):
    batch = {k:[ex[k] for ex in examples] for k in examples[0].keys()}
    bs = len(examples)
    collated = {}
    
    for key in batch.keys():
        collated[key] = pad(batch[key],key,pad_tokens[key])
        
    return collated

In [31]:
collate_fn(examples)

{'input_ids': tensor([[    0,   313, 25740,  ...,     1,     1,     1],
         [    0, 24784, 18106,  ...,     1,     1,     1],
         [    0, 26550, 43309,  ...,     1,     1,     1],
         [    0, 24784, 18106,  ...,     1,     1,     1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'special_tokens_mask': tensor([[1, 0, 0,  ..., 1, 1, 1],
         [1, 0, 0,  ..., 1, 1, 1],
         [1, 0, 0,  ..., 1, 1, 1],
         [1, 0, 0,  ..., 1, 1, 1]]),
 'bbox': tensor([[[  0,   0,   0,   0],
          [ 75,  88, 119,  99],
          [ 75,  88, 119,  99],
          ...,
          [  0,   0,   0,   0],
          [  0,   0,   0,   0],
          [  0,   0,   0,   0]],
 
         [[  0,   0,   0,   0],
          [383, 314, 449, 327],
          [383, 314, 449, 327],
          ...,
          [  0,   0,   0,   0],
          [  0,   0,   0,   0],
          [  0,   0,   0,   0]]

In [35]:
max(5,10)

10